In [ ]:
import main
import pandas as pd
import json
import logging
import util
import process


for lib in ('flower', 'matplotlib'):
    logging.getLogger(lib).setLevel(level=logging.WARN)
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)


In [ ]:
# Configure your run here

params = ['param.batchsize=32',
          'param.batches=128',
          'param.epsilon=0.001',
          'param.runs=10',
          'param.epochs=1',
          'param.top_j=2',
          'optimizer=SGD',
          'dataset=MNIST',
          'eval.threshold=0.1',
          
          #'runtime.output_dir=./output', # Where to save the results
          #'runtime.dataset_dir=./datasets', # Where to load the datasets
         ]

args = util.get_config(params)



The dataset will be downloaded if not available in the current directory. When performing the experiment on `CelebA`, the data might need to be downloaded manually due to a [bug](https://github.com/pytorch/vision/issues/2262).

In [ ]:
# Run training
main.main(args)

In [ ]:
# Plot resulting ROC curve
util.plot_roc(args)

In [ ]:
# Process all results for everything done so far
results_json = process.main(args)
# or use data obtained when producing the paper
#results_json = 'results.orig.json' 
with open(results_json, 'r') as fp:
    results = json.load(fp)
df = util.dict_to_table(results)

You can use pandas to display the results of various configurations (in variable `df`).
Pandas [indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) can be used as showcased.

The following columns are available:

| Name    | Content |
| :---     | :---     |
| Batches | Number of batches per epoch |
| Batch size | Size of each batch |
| Ep. | Number of epochs |
| Opt. | Optimizer (SGD or Adam) |
| Dataset | MNIST, CIFAR10, CIFAR100 or CelebA |
| M | Number of values chose for the attack |
| epsilon | Constant bias selected for the output neuron |
| runs | Number of runs conducted (for each t=0/1) |
| FPR/FNR/Acc. | False Negative Rate, False Positive Rate and Accuracy with xi as chose above |
| AUC | Area under the receiver operating characteristic curve |


In [ ]:
selector =   (df['Opt.']    == args.param.optimizer) \
           & (df['Dataset'] == args.param.dataset) \
           & (df['Ep.']     == args.param.epochs)

df[selector]